In [1]:
import numpy as np
import os
import scipy.io as sio
import pandas as pd

In [2]:
gt=np.load(r"/media/eadu/DATA/sim_groudtruth/gt_in_slices_repeat/gt_model2_scatter1_frame4.npy")
print(gt.shape)
gta = gt[0]
t= sio.loadmat(r"/media/eadu/DATA/fusionNet/displacementBYtradMethod/sim_result/model2scatter_1_frame4.mat")['displ']
print(t.shape)
ta=t[0]
o = np.load(r"/media/eadu/DATA/displacementBYoptflow/displ_model2_scatter1_frame4.npy")
oa=o[1]
print(o.shape)



(2, 1968, 226)
(2, 1968, 226)
(2, 2048, 256)


In [ ]:
class Decoder(nn.Module):
    def __init__(self, out_channels, in_channels=1024):
        super(Decoder, self).__init__()
        # self.conv_in = nn.Conv2d(in_channels, 512, kernel_size=3, stride=1, padding=1)
        self.upconv1 = nn.Conv2d(in_channels, 512, kernel_size=3, stride=1, padding=1)
        self.upconv2 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1)
        self.upconv3 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1)
        # self.upconv4 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.final_conv = nn.Conv2d(128, out_channels, kernel_size=7, stride=1, padding=3)
    
    def forward(self, x4, x3, x2, x1):
        # x4 = F.relu(self.conv_in(F.interpolate(x4, scale_factor=2, mode='nearest')))
        x = F.relu(self.upconv1(F.interpolate(x4, scale_factor=2, mode='nearest')))
        if x.shape != x3.shape:
            x = F.interpolate(x, size=x3.shape[-2:], mode='bilinear', align_corners=False)
        
        x = F.relu(self.upconv2(F.interpolate(x + x3, scale_factor=2, mode='nearest')))
        if x.shape != x2.shape:
            x = F.interpolate(x, size=x2.shape[-2:], mode='bilinear', align_corners=False)
        
        x = F.relu(self.upconv3(F.interpolate(x + x2, scale_factor=2, mode='nearest')))
        if x.shape != x1.shape:
            x = F.interpolate(x, size=x1.shape[-2:], mode='bilinear', align_corners=False)        
        # x = F.relu(self.upconv4(F.interpolate(x + x1, scale_factor=2, mode='nearest')))
        x = torch.tanh(self.final_conv(x + x1))
        return x

In [ ]:
class Decoder(nn.Module):
    def __init__(self,out_channels,in_channels=1024):
        super(Decoder, self).__init__()
        # self.conv_in = nn.Conv2d(in_channels, 512, kernel_size=3, stride=1, padding=1)
        self.upconv1 = nn.Conv2d(in_channels, 512, kernel_size=3, stride=1, padding=1)
        self.upconv2 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1)
        self.upconv3 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1)
        # self.upconv4 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1)
        self.final_conv = nn.Conv2d(128, out_channels, kernel_size=7, stride=1, padding=3)
    
    def forward(self, x4, x3, x2, x1):
        # x4 = F.relu(self.conv_in(F.interpolate(x4, scale_factor=2, mode='nearest')))
        x = F.relu(self.upconv1(F.interpolate(x4, scale_factor=2, mode='nearest')))
        if x.shape != x3.shape:
            x = F.interpolate(x,size=x3.shape[-2:], mode='bilinear', align_corners=False)
        
        x = F.relu(self.upconv2(F.interpolate(x + x3, scale_factor=2, mode='nearest')))
        if x.shape != x2.shape:
            x = F.interpolate(x,size=x2.shape[-2:], mode='bilinear', align_corners=False)
        
        x = F.relu(self.upconv3(F.interpolate(x + x2, scale_factor=2, mode='nearest')))
        if x.shape != x1.shape:
            x = F.interpolate(x,size=x1.shape[-2:], mode='bilinear', align_corners=False)        
        # x = F.relu(self.upconv4(F.interpolate(x +x1, scale_factor=2, mode='nearest')))
        x = torch.sigmoid(self.final_conv(x+x1))
        return x